# 1D Bar - Isogeometric Analysis
## Analysis - What happens under the hood?
*Mathematical Formulation of IGA, Derivation and Assembly of Stiffness Matrix while solving a simple 1D Bar Example*

### Governing Equation - Strong vs Weak Formulation
The governing differential equation for the static structural problem of the 1D Bar is given by:

$$Δu(x) = -f(x)$$

where $Δ$ represents the Laplacian operator. This equation is well known as the Poisson's equation. 

By expanding the laplacian for the 1D Bar problem we end up with the following:

$$\frac{\partial ^ 2u(x)}{\partial x^2} = -f(x)$$

This is known as the "Strong form" of the poisson's equation. This equation can provide the exact solution for particular (finite) points inside the domain or for particular points at the boundaries, caclulated by solving the respective equations for the boundaries (they can be Dirichlet, Neumann or Robin). However, both the domain and boundaries are described by an infinite number of point and in between the selected/particular points for the exact solution of the strong form equations we will end up probably with oscillations, unveiling a huge drawback of the strong form methods. To provide a control between the selected points of the equations the Weak Formulation is recommended instead. As an example we could apply an averaging technique between the particular points of the domain, which is somewhat the case in Finite Element Analysis (FEA) as well. We will not cover the whole derivation procedure for the Weak Formulation of the poisson's equation but the basic idea behind it is that we multiply the strong form by a test function (the so-called weight function) and integrate over the domain. The test functions are chosen specifically for each problem.

### General Weak Formulation with Dirichlet Boundary Conditions
So, returning back to our methodology, to derive the weak formulation we multiply the strong form by a test function v(x) and integrate over the domain:

$$
\int_Ω  \frac{\partial ^ 2u(x)}{\partial x^2}v(x) dΩ = - \int_Ω f(x)v(x) dΩ
$$

By applying the integration by parts (Green's Theorem) to the left-hand side to move the derivative from $u(x)$ to the test function $v(x)$:

$$
\int_{a}^{b} \frac{\partial^2 u(x)}{\partial x^2}v(x) dx = \left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{a}^{b} - \int_{a}^{b} \frac{\partial u(x)}{\partial x} \frac{\partial v(x)}{\partial x} dx 
$$

Here the term $\left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{a}^{b}$ is the boundary term evaluated at the boundaries of the domain a and b. For the domain $Ω = [0,1]$ this becomes:

$$
\left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{0}^{1} - \int_{0}^{1} \frac{\partial u(x)}{\partial x} \frac{\partial v(x)}{\partial x} \, dx 
$$

Assuming Dirichlet boundary conditions, where $u(0) = u(1) = 0$ we also consider $v(0)=v(1)=0$. This simplifies the boundary term to zero because the test function v(x) is zero at the boundaries:

$$
\left[ \frac{\partial u(x)}{\partial x} v(x) \right]_{0}^{1} = 0
$$

Thus, the weak form of the equation reduces to:
$$
\int_{0}^{1} \frac{\partial u(x)}{\partial x} \frac{\partial v(x)}{\partial x} \, dx = \int_{0}^{1} f(x)v(x)\, dx
$$

### Handling Boundary Conditions
In the weak form, the boundary conditions are naturally incorporated through the choice of test functions. For Dirichlet boundary conditions, the test functions are chosen such that they are zero at the boundaries. This ensures that the boundary terms vanish, simplifying the formulation. For Neumann or Robin boundary conditions, the boundary term would not vanish and would need to be included in the weak form. The weak form provides a flexible and stable way to incorporate these conditions and solve the problem numerically.

> This can also be found as:
>$$
\int_Ω \nabla u(x) \nabla v(x) \, dΩ - \int_Ω \frac{\partial u}{\partial n} \, dΓ = \int_Ω f(x)v(x) \, dΩ
>$$
>Given that the boundary condition v(x) = 0 on ${\partial Ω}$:
>$$
\int_Ω \nabla u(x) \nabla v(x) \, dΩ = \int_Ω f(x)v(x) \, dΩ
>$$

In the context of the finite element method and isogeometric analysis, the choice of approximating the solution $u(x)$ and the test function $v(x)$ using a finite set of basis function Ni(x) and Nj(x), respectively, is grounded in the method of weighted residuals and the principle of Galerkin's method.

### Weighted Residuals and Galerkin's Method
The primary goal in numerical methods for partial differential equations (PDEs) is to approximate the continuous solution $u(x)$ by a discrete function that satisfies the PDE in an average sense over the domain. This is achieved through the method of weighted residuals, where the residual (the error in satisfying the PDE) is minimized in an integral sense.

#### Weighted Residuals
The idea behind the weighted residuals method is to convert the differential equation into an integral form where the residuals (errors) are weighted and forced to be zero over the domain. The residual $R(u)$ of the differential equation is given by:

$$
R(u) = - \frac{\partial^2u(x)}{\partial x^2} - f(x)
$$

To ensure that the residuals are minimized, we integrate the residual multiplied by a set of weighting functions $v(x)$ over the domain:
$$
\int_Ω R(u)v(x)\, dΩ = 0
$$

#### Galerkin's Method

Bubnov-Galerkin's method is a specific case of the weighted residuals method where the weighting functions are chosen to be the same as the basis functions used to approximate the solution. This means we use the basis functions Nj(x) as our weighting functions:
$$
\int_Ω (- \frac{\partial^2u(x)}{\partial x^2} - f(x))Nj(x)\, dΩ = 0
$$

We saw previously the technique for eliminating the second derivative by intergating by parts, thus by applying the same principle for the enclosed in parentheses term we end up with:

$$
\int_Ω \frac{\partial^2u(x)}{\partial x^2} Nj(x) dΩ = -\int_Ω \frac{\partial u(x)}{\partial x} \frac{\partial Νj(x)}{\partial x} dΩ
$$

Substituting this into the weighted residuals equation we get:
$$
\int_Ω \nabla u(x) \nabla Nj(x)\, dΩ = \int_Ω f(x)Nj(x)\, dΩ
$$

### Approximation of $u(x)$, $v(x)$ using Basis Functions

Now, we approximate the solution u(x) as a linear combination of basis functions Ni(x)
$$
u(x) = \sum_{i=Ι} u(i)Ni(x) 
$$
and
$$
v(x) = Nj(x)
$$
ui: in case of Isogeometric Analysis these coefficients corresponds to the control points. These control points are the unknowns that we solve for in the system of equations. Explanation
In IGA, the geometry of the domain and the approximation of the solution are both represented using the same set of basis functions (e.g., B-splines or NURBS). The process involves the following steps:

Geometry Representation: The physical geometry of the domain is defined using control points and basis functions. For example, the shape of the domain is represented as a combination of basis functions and their corresponding control points in the geometric space.
Solution Approximation: The approximate solution $u(x)$ is expressed as a linear combination of the same basis functions used for geometry, but with a different set of control points that represent the solution values.

Thus, the control points in the solution approximation are analogous to nodal values in the traditional finite element method (FEM). They represent the values of the solution at specific locations in the parametric domain, which are determined by solving the system of equations derived from the weak form of the governing differential equations.

Here’s how it all ties together:

The basis functions Ni(x) are predefined and depend on the choice of the knot vector and the degree of the basis functions.
The control points $ui$ are the unknowns that we need to determine. They scale the basis functions to construct the approximate solution
$u(x)$.

### Formulating the System of Equations
Substituting the approximation into the weak form, we get:
$$
\int_Ω  \nabla (\sum_{i=I} uiNi(x)) \nabla Nj(x) dΩ = \int_Ω f(x)Nj(x)dΩ
$$

By linearity of the integral and the dot product, this simplifies to:
$$
\sum_{i=I}ui \int_Ω \nabla ( Ni(x)) \nabla Nj(x) dΩ = \int_Ω f(x)Nj(x)dΩ
$$

This leads to a system of linear equations for the coefficients ui:
$$
\sum_{i=I}Kijui = Fj
$$
where
$$
Kij = \int_Ω \nabla Ni(x) \nabla Nj(x) dΩ
$$
$$
Fj = \int_Ω f(x)Nj(x)dΩ
$$

In matrix form we write: $Ku = F$.

---
## Problem Description

Consider a bar of Length L = 1 m, subjected to a uniform load $f(x) = 1$ unit per length. The bar is fixed at both ends with Dirichlet boundary conditions: $u(0) = u(1) = 0$. Solve the above problem using the Isogeometric Analysis with a discretization of 2 elements and B-Spline basis functions.

## Construct the basis functions

Knot vector:  
$$ξ = [0,0,0,0.5,1,1,1]$$
___
>For degree 0, the basis functions are defined as:
>$$
N_{i,0}(ξ) =
\begin{cases}
1 & \text{if } \xi_i \leq \xi < \xi_{i+1} \\
0 & \text{otherwise}
\end{cases}
>$$


For $ξ_0 = 0$ and $ξ_1 = 0$

$$
N_{0,0}(ξ) = 
\begin{cases}
1 & \text{if } 0 \leq \xi < 0 \\
0 & \text{otherwise}
\end{cases}
$$

So: $N_{0,0}(ξ) = 0$


For $ξ_1 = 0$ and $ξ_2 = 0$

$$
N_{1,0}(ξ) = 
\begin{cases}
1 & \text{if } 0 \leq \xi < 0 \\
0 & \text{otherwise}
\end{cases}
$$
So: $N_{1,0}(ξ) = 0$


For $ξ_2 = 0$ and $ξ_3 = 0.5$

$$
N_{2,0}(ξ) = 
\begin{cases}
1 & \text{if } 0 \leq \xi < 0.5 \\
0 & \text{otherwise}
\end{cases}
$$


For $ξ_3 = 0.5$ and $ξ_4 = 1$

$$
N_{3,0}(ξ) = 
\begin{cases}
1 & \text{if } 0.5 \leq \xi < 1 \\
0 & \text{otherwise}
\end{cases}
$$


For $ξ_4 = 1$ and $ξ_5 = 1$

$$
N_{4,0}(ξ) = 
\begin{cases}
1 & \text{if } 1 \leq \xi < 1 \\
0 & \text{otherwise}
\end{cases}
$$
So: $N_{4,0}(ξ) = 0$


For $ξ_5 = 1$ and $ξ_6 = 1$

$$
N_{5,0}(ξ) = 
\begin{cases}
1 & \text{if } 1 \leq \xi < 1 \\
0 & \text{otherwise}
\end{cases}
$$
So: $N_{5,0}(ξ) = 0$

___
>For degree 1, the basis functions are defined as:
>$$
N_{i,1}(ξ) = \frac{ξ - ξ_i}{ξ_{i+1} - ξ_{i}} Ν_{i,0}(ξ) + \frac{ξ_{i+2} - ξ}{ξ_{i+2} - ξ_{i+1}}Ν_{i+1, 0}(ξ)
>$$

Calculate $N_{0,1}$ :

$ξ_{0} = 0$, $ξ_{1} = 0$, $ξ_{2} = 0$

$$
Ν_{0,1}(ξ) = \frac{ξ-ξ_0}{ξ_1-ξ_0}Ν_{0,0}(ξ) + \frac{ξ_2-ξ}{ξ_2-ξ_1}Ν_{1,0}(ξ)
$$

$$
Ν_{0,1}(ξ) = \frac{ξ-0}{0-0}0 + \frac{0-ξ}{0-0}0
$$

Since both terms involve division by zero, we define generally: $N_{0,1}(ξ) = 0$.


Calculate $N_{1,1}$ :

$ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0.5$

$$
Ν_{1,1}(ξ) = \frac{ξ-ξ_1}{ξ_2-ξ_1}Ν_{1,0}(ξ) + \frac{ξ_3-ξ}{ξ_3-ξ_2}Ν_{2,0}(ξ)
$$

$$
Ν_{1,1}(ξ) = \frac{ξ-0}{0-0}0 + \frac{0.5-ξ}{0.5-0}Ν_{2,0}(ξ)
$$

$$
Ν_{1,1}(ξ) = \frac{0.5-ξ}{0.5}Ν_{2,0}(ξ)
$$

$$
Ν_{1,1}(ξ) = (1-2ξ)Ν_{2,0}(ξ)
$$

By substituting $N_{2,0}$ from above:

$$
N_{1,1}(ξ) = 
\begin{cases}
1 - 2ξ & \text{if } 0 \leq \xi < 0.5 \\
0 & \text{otherwise}
\end{cases}
$$

Calculate $N_{2,1}$ :

$ξ_{2} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$

$$
Ν_{2,1}(ξ) = \frac{ξ-ξ_2}{ξ_3-ξ_2}Ν_{2,0}(ξ) + \frac{ξ_4-ξ}{ξ_4-ξ_3}Ν_{3,0}(ξ)
$$

$$
Ν_{2,1}(ξ) = \frac{ξ-0}{0.5-0}N_{2,0}(ξ) + \frac{1-ξ}{1-0.5}Ν_{3,0}(ξ)
$$

$$
Ν_{2,1}(ξ) = 2ξ Ν_{2,0}(ξ) +(2-2ξ)Ν_{3,0}(ξ) 
$$

By substituting $N_{2,0}$ and $N_{3,0}$ from above:

$$
N_{2,1}(ξ) = 
\begin{cases}
2ξ & \text{if } 0 \leq \xi < 0.5 \\
2-2ξ & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

Calculate $N_{3,1}$ :

$ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$

$$
Ν_{3,1}(ξ) = \frac{ξ-ξ_3}{ξ_4-ξ_3}Ν_{3,0}(ξ) + \frac{ξ_5-ξ}{ξ_5-ξ_4}Ν_{4,0}(ξ)
$$

$$
Ν_{3,1}(ξ) = \frac{ξ-0.5}{1-0.5}N_{3,0}(ξ) + \frac{1-ξ}{0-0}Ν_{4,0}(ξ)
$$

$$
Ν_{3,1}(ξ) = (2ξ-1)Ν_{3,0}(ξ) 
$$

By substituting $N_{3,0}$ from above:

$$
N_{3,1}(ξ) = 
\begin{cases}
2ξ-1 & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$


Calculate $N_{4,1}$ :

$ξ_{4} = 1$, $ξ_{5} = 1$, $ξ_{6} = 1$

$$
Ν_{4,1}(ξ) = \frac{ξ-ξ_4}{ξ_5-ξ_4}Ν_{4,0}(ξ) + \frac{ξ_6-ξ}{ξ_6-ξ_5}Ν_{5,0}(ξ)
$$

$$
Ν_{4,1}(ξ) = \frac{ξ-1}{1-1}N_{4,0}(ξ) + \frac{1-ξ}{1-1}Ν_{5,0}(ξ)
$$

$$
Ν_{4,1}(ξ) = 0 
$$

___
>For degree 2, the basis functions are defined as:
>$$
N_{i,2}(ξ) = \frac{ξ - ξ_i}{ξ_{i+1} - ξ_{i}} Ν_{i,1}(ξ) + \frac{ξ_{i+2} - ξ}{ξ_{i+2} - ξ_{i+1}}Ν_{i+1, 1}(ξ)
>$$

Calculate $N_{0,2}$ :

$ξ_{0} = 0$, $ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0.5$

$$
Ν_{0,2}(ξ) = \frac{ξ-ξ_0}{ξ_2-ξ_0}Ν_{0,1}(ξ) + \frac{ξ_3-ξ}{ξ_3-ξ_1}Ν_{1,1}(ξ)
$$

$$
Ν_{0,2}(ξ) = 2ξΝ_{0,1}(ξ) + 2(0.5-ξ)Ν_{1,1}(ξ)
$$

Given the basis functions N_{0,1}(ξ) = 0 and Ν_{1,1}(ξ):
$$
N_{1,1}(ξ) = 
\begin{cases}
1 - 2ξ  & \text{if } 0 \leq \xi < 0.5 \\
0           & \text{otherwise}
\end{cases}
$$

$$
N_{0,2}(ξ) = 
\begin{cases}
(1 - 2ξ)^2  & \text{if } 0 \leq \xi < 0.5 \\
0           & \text{otherwise}
\end{cases}
$$

Calculate $N_{1,2}$ :

$ξ_{1} = 0$, $ξ_{2} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$

$$
Ν_{1,2}(ξ) = \frac{ξ-ξ_1}{ξ_3-ξ_1}Ν_{1,1}(ξ) + \frac{ξ_4-ξ}{ξ_4-ξ_2}Ν_{2,1}(ξ)
$$

$$
Ν_{1,2}(ξ) = \frac{ξ-0}{0.5-0}Ν_{1,1}(ξ) + \frac{1-ξ}{1-0}Ν_{2,1}(ξ)
$$

$$
Ν_{1,2}(ξ) = 2ξΝ_{1,1}(ξ) + (1-ξ)Ν_{2,1}(ξ)
$$

From the above calculation we know that:
$$
N_{1,1}(ξ) = 
\begin{cases}
1 - 2ξ & \text{if } 0 \leq \xi < 0.5 \\
0 & \text{otherwise}
\end{cases}
$$

$$
N_{2,1}(ξ) = 
\begin{cases}
2ξ & \text{if } 0 \leq \xi < 0.5 \\
2-2ξ & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
2ξ(1-2ξ) + (1-ξ)2ξ  & \text{if } 0 \leq \xi < 0.5 \\
(1-ξ)(2-2ξ)         & \text{if } 0.5 \leq \xi < 1 \\    
0                   & \text{otherwise}
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
4ξ - 6ξ^2       & \text{if } 0 \leq \xi < 0.5 \\
2 - 4ξ + 2ξ^2   & \text{if } 0.5 \leq \xi < 1 \\    
0               & \text{otherwise}
\end{cases}
$$

Calculate $N_{2,2}$ :

$ξ_{2} = 0$, $ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$

$$
Ν_{2,2}(ξ) = \frac{ξ-ξ_2}{ξ_4-ξ_2}Ν_{2,1}(ξ) + \frac{ξ_5-ξ}{ξ_5-ξ_3}Ν_{3,1}(ξ)
$$

$$
Ν_{2,2}(ξ) = \frac{ξ-0}{1-0}Ν_{2,1}(ξ) + \frac{1-ξ}{1-0.5}Ν_{3,1}(ξ)
$$

$$
Ν_{2,2}(ξ) = ξΝ_{2,1}(ξ) + 2(1-ξ)Ν_{3,1}(ξ)
$$

Given that: 
$$
N_{2,1}(ξ) = 
\begin{cases}
2ξ & \text{if } 0 \leq \xi < 0.5 \\
2-2ξ & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{3,1}(ξ) = 
\begin{cases}
2ξ-1 & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
ξ 2ξ                      & \text{if } 0 \leq \xi < 0.5 \\
ξ(2-2ξ) + 2(1-ξ)(2ξ-1)    & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
2ξ^2             & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^2 + 6ξ -2    & \text{if } 0.5 \leq \xi < 1 \\
0                & otherwise
\end{cases}
$$

Calculate $N_{3,2}$ :

$ξ_{3} = 0.5$, $ξ_{4} = 1$, $ξ_{5} = 1$, $ξ_{6} = 1$

$$
Ν_{3,2}(ξ) = \frac{ξ-ξ_3}{ξ_5-ξ_3}Ν_{3,1}(ξ) + \frac{ξ_6-ξ}{ξ_6-ξ_4}Ν_{4,1}(ξ)
$$

$$
Ν_{3,2}(ξ) = \frac{ξ - 0.5}{1 - 0.5}Ν_{3,1}(ξ) + \frac{1 - ξ}{1 - 1}Ν_{4,1}(ξ)
$$

$$
Ν_{3,2}(ξ) = \frac{ξ - 0.5}{0.5}Ν_{3,1}(ξ)
$$

$$
Ν_{3,2}(ξ) = (2ξ - 1)Ν_{3,1}(ξ)
$$

Given that:

$$
N_{3,1}(ξ) = 
\begin{cases}
2ξ-1 & \text{if } 0.5 \leq \xi < 1 \\
0 & otherwise
\end{cases}
$$

$$
N_{3,2}(ξ) = 
\begin{cases}
(2ξ-1)^2    & \text{if } 0.5 \leq \xi < 1 \\
0           & otherwise
\end{cases}
$$

___
Summarize the p = 2 Basis Functions:

$$
N_{0,2}(ξ) = 
\begin{cases}
(1 - 2ξ)^2  & \text{if } 0 \leq \xi < 0.5 \\
0           & \text{otherwise}
\end{cases}
$$

$$
N_{1,2}(ξ) = 
\begin{cases}
4ξ - 6ξ^2       & \text{if } 0 \leq \xi < 0.5 \\
2 - 4ξ + 2ξ^2   & \text{if } 0.5 \leq \xi < 1 \\    
0               & \text{otherwise}
\end{cases}
$$

$$
N_{2,2}(ξ) = 
\begin{cases}
2ξ^2             & \text{if } 0 \leq \xi < 0.5 \\
-6ξ^2 + 6ξ -2    & \text{if } 0.5 \leq \xi < 1 \\
0                & otherwise
\end{cases}
$$

$$
N_{3,2}(ξ) = 
\begin{cases}
(2ξ-1)^2    & \text{if } 0.5 \leq \xi < 1 \\
0           & otherwise
\end{cases}
$$

Calculate the derivatives of the Basis Functions:
$$
\frac {d}{dx} N_{0,2}(ξ) = 
\begin{cases}
8ξ - 4  & \text{if } 0 \leq \xi < 0.5 \\
0       & \text{otherwise}
\end{cases}
$$

$$
\frac {d}{dx} N_{1,2}(ξ) = 
\begin{cases}
4 - 12ξ   & \text{if } 0 \leq \xi < 0.5 \\
-4 + 4ξ   & \text{if } 0.5 \leq \xi < 1 \\    
0         & \text{otherwise}
\end{cases}
$$

$$
\frac {d}{dx} N_{2,2}(ξ) = 
\begin{cases}
4ξ            & \text{if } 0 \leq \xi < 0.5 \\
-12ξ^2 + 6    & \text{if } 0.5 \leq \xi < 1 \\
0             & otherwise
\end{cases}
$$

$$
\frac {d}{dx} N_{3,2}(ξ) = 
\begin{cases}
8ξ - 4    & \text{if } 0.5 \leq \xi < 1 \\
0         & otherwise
\end{cases}
$$
___
### Calculate the Element's Stiffness Matrix

#### Stiffness Matrix in Physical Space
As we saw before, the stiffness matrix $K$ in the physical space is given by:

$$
K_{ij} = \int_Ω \nabla N_{i}(x) \nabla N_{j}(x)\, dΩ
$$

#### Transformation to the parametric space:
##### Mapping Function
For a 1D element, the physical coordinate $x$ is related to the parametric coordinate $ξ$ via a linear mapping:
$x = a + bξ$
where $a$ and $b$ are the start and length of the element in physical space. For an element $e$ with nodes at $x_{e}$ and $x_{e+1}$, this becomes:
$x = x_e + (x_{e+1} - x_e)ξ$
##### Jacobian Determinant
The Jacobian $J$ is the derivative of the mapping function:
$J = \frac{dx}{dξ} = x_{e+1} - x_{e}$
For each element $e$, this is constant and equals the lenght of the element, so for this case $J = 0.5$ as we have 2 elements.
##### Transform the Integral
So now we have the stiffness matrix in the physical space given as:
$$
K_{ij} = \int_{x_e}^{x_{e+1}} \nabla N_{i}(x) \nabla N_{j}(x) dx
$$
Substitute x = x_{e} + (x_{e+1} - x_e)ξ:
$$
dx = (x_{e+1} - x_e)dξ
$$
So the stiffness matrix becomes:
$$
K_{ij} = \int_{0}^{1} \nabla N_{i}(x(ξ)) \nabla N_{j}(x(ξ))(x_{e+1} - x_
{e}) dξ
$$
where $\nabla N_{i}(x(ξ))$ and $\nabla N_{j}(x(ξ))$ are the derivatives of the basis functions evaluated at the parametric coordinates.

##### Gaussian Quadrature to calculate the Integral
But what is the suggested porcedure for calculating the integral of the stiffness matrix $K$?
___
*Gaussian quadrature* is a numerical integration technique used to approximate the integral of a function. It is particularly useful for integrals that are difficult to evaluate analytically. The method uses a set of weighted sample points (or nodes) to approximate the integral. Let's discuss this method in here for a quick recap.

For a given quadrature rule, we have a set of quadrature points ξ_{Κ} and weights W_{K}:
$$
\int_Ω f(ξ) dΩ \approx \sum_{K} f(ξ_{Κ})W_{K}
$$
So the integral is approximated by the sum of products of the function evaluated at the Gauss points times the respective weight.

For the 1D integration, Gaussian quadrature with 2 points is defined as:
$$
\int_{-1}^{1} f(x)dx = w_1f(x_1) + w_2f(x_2)
$$

This equation is exact for $f(x) = $ $1$, $x$, $x^2$ and $x^3$. Let's examine each case:

$$f(x) = 1 => 2 = w_1 + w_2$$
$$f(x) = x => 0 = w_1x_1 + w_2x_2$$
$$f(x) = x^2 => 2/3 = w_1x_1^2 + w_2x_2^2$$
$$f(x) = x^3 => 0 = w_1x_1^3 + w_2x_2^3$$

$$x_1 = - x_2$$
$$w_1 = w_2$$

$$w_1 = w_2 = 1$$
$$x1 = -1/\sqrt3$$ 
$$x2 = 1/\sqrt3$$
___

So finally, with the utilization of Gaussian Quadrature the transformed stiffness matrix to the parametric domain is now given by the following sum:
$$
K_{ij} = \int_Ω \nabla N_i(ξ) \nabla N_j(ξ) det(J(ξ))dΩ
$$
$$
K_{ij} = \sum_{K} \nabla N_i(ξ_K) \nabla N_j(ξ_K) det(J(ξ_K))w_{K}
$$
where $K$ is the Gauss Point $J(ξ)$ is the Jacobian matrix of the transformation from parametrix coordinates $ξ$ to physical coordinates $x$. The Jacobian matrix relates the derivatives in the parametric space to the physical space and the typical Jacobian from $(x, y)$ to $(ξ_1, ξ_2)$ is given by:
$$
\begin{equation}
J =
\begin{bmatrix}
\frac{\partial x}{\partial \xi_1} & \frac{\partial x}{\partial \xi_2} \\
\frac{\partial y}{\partial \xi_1} & \frac{\partial y}{\partial \xi_2}
\end{bmatrix}
\end{equation}
$$
The determinant of the Jacobian $det(J(ξ))$ gives the area or volume scale factor.
##### Jacobian Determinant for 1D Bar
The Jacobian $J$ is the derivative of the mapping function:
$J = \frac{dx}{dξ} = x_{e+1} - x_{e}$
For each element $e$, this is constant and equals the lenght of the element, so for this case $J = 0.5$ as we have 2 elements.

##### Strain-Displacement Matrix
What is it, why is it introduced and will i use it in 1D ?




TO_DO:
-2D hand calculation ?
-Examine the theory of assemblying the stiffness matrix based on the PhD thesis provided.
Optional:
-Provide a python script for this solutions